In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## 1.모델 가져오기(git clone)

In [ ]:
%pwd

'/content'

In [ ]:
%cd /content/drive/MyDrive/yolo/hard hat detection

/content/drive/MyDrive/yolo/hard hat detection


In [ ]:
%pwd

'/content/drive/MyDrive/yolo/hard hat detection'

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd /content/drive/MyDrive/yolo/hard hat detection/yolov5

/content/drive/MyDrive/yolo/hard hat detection/yolov5


In [ ]:
!pip install -r requirements.txt
import torch
from IPython.display import Image, clear_output
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## 2.데이터셋 다운로드

In [ ]:
%pwd

'/content'

In [ ]:
%cd /content/drive/MyDrive/yolo/hard hat detection/yolov5/hard hat
%pwd

In [ ]:
!curl -L "https://public.roboflow.com/ds/CIlvgLTvGm?key=iDkCxkWy0I" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
from glob import glob
train_img_list = glob("train/images/*")
test_img_list = glob("test/images/*")
print(len(train_img_list), len(test_img_list))

In [ ]:
import yaml
with open("/content/drive/MyDrive/yolo/hard hat detection/yolov5/hard hat/train.txt", 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')
with open("/content/drive/MyDrive/yolo/hard hat detection/yolov5/hard hat/test.txt", 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

## 3.yaml파일 만들기

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line,cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%cat data.yaml

In [ ]:
%%writetemplate data.yaml

train: ./train/images
val: ./test/images

nc: 3
names: ['head', 'helmet', 'person']

In [ ]:
%cat data.yaml

## 4.FineTunning

In [ ]:
with open("data.yaml", 'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])
  print(num_classes)

In [ ]:
%pwd

In [ ]:
%cd /content/drive/MyDrive/yolo/hard hat detection/yolov5/models
%pwd

In [ ]:
%cat yolov5s.yaml

In [ ]:
%%writetemplate custom_yolov5s.yaml

# Parameters
nc: {num_classes} # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]

## 5.Train

In [ ]:
%pwd

'/content/drive/MyDrive/yolo/hard hat detection/yolov5/models'

In [ ]:
%cd /content/drive/MyDrive/yolo/hard hat detection/yolov5
%pwd

/content/drive/MyDrive/yolo/hard hat detection/yolov5


'/content/drive/MyDrive/yolo/hard hat detection/yolov5'

In [ ]:
!python train.py --img 640 --batch 32 --epochs 100 --data ./hardhat/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name hard hat_result --cache

In [ ]:
Image(filename="", width=600)

In [ ]:
!python val.py --weights /content/drice/MyDrive/yolo/yolov5/runs/train/pothole_result/weights/best.pt --data /content/drice/MyDrive/yolo/yolov5/pothole/data.yaml --img 640 --iou 0.65 --half